## *NOTE* Frequency indices T?90, R??mm, N??? have errors in them. Output files deleted.

Calculate the climatology and anomalies for each index.  
Then calculate the statistical significance to plot results

Using ARISE-SAI runs with 5% seeding. Because only 5 runs of SSP245 have tmax and tmin, subtract the ensemble mean climatology from all temperatures.

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np # data arrays
import xarray as xr # data array manipulation
import pandas as pd
import os
import glob # to enable pattern matching
from scipy import stats

In [4]:
iDir = "/glade/work/maritye/Data/ARISE-MCB/ETCCDI/"
inSSP = '/glade/work/maritye/Data/ARISE-SAI/ETCCDI/SSP245/'
oDir = '/glade/work/maritye/Data/ARISE-MCB/ETCCDI/Anomalies/'

varnam = ['PRECT', 'TREFHTMX', 'TREFHTMN']

Can't quite work out how to loop the directories and the separate lists of indices. So just do three repeated calculations for anomalies and significances

Create standard information to include with each data set

In [77]:
years = np.arange(2035,2070)
dates= '2035-2069'
member=range(10)
lat = np.linspace(-90,90,num=192)
lon = np.linspace(0,358.75,num=288)
dims = ('member', 'lat', 'lon')
coords = dict(member = member, lat=lat, lon=lon)
attribs = dict(description='Extreme Index Anomalies based on ETCCDI definitions. MCB scenarios with 5% global seeding', 
                history='Created by Mari Tye November 2023.' ),


In [108]:
pindices = ['CDD','CWD','RX1D','N95','N99','R20mm','RX5D','PRCPTOT', 'SDII','R10mm'] # something funny with P95 and P99 ignore these 2 for MCB runs
txindices = ['TXX',  'TXN', 'TX10', 'TX90'] #calculated the fixed threshold indices but they aren't very meaningful at a global scale so not including in paper or significance tests.
tnindices = ['TNX', 'TNN', 'TN10', 'TN90']

Precipitation anomalies and significance of changes

In [65]:
for i in range(len(pindices)):
    print(pindices[i])
    
    # this creates a vector, but only want first element to use as file name
    praw_ssp = glob.glob(inSSP + varnam[0] + '/*' + pindices[i] + '*.nc')[0]
    praw_mcb = glob.glob(iDir + varnam[0] + '/*' + pindices[i] + '*.nc')[0]
    
    with xr.open_dataset(praw_ssp) as ds_ssp:
        cont = ds_ssp.sel(year=slice('2019-01-01','2039-12-31'))
        ssp_ec = ds_ssp.sel(year=slice('2049-01-01', '2069-12-31'))
        clim = cont.mean(dim='year')
        
    with xr.open_dataset(praw_mcb) as ds_mcb:
        mcb_ec = ds_mcb.sel(year=slice('2049-01-01', '2069-12-31'))
        
    
    
    bc_anom = cont - clim
    ssp_anom = ssp_ec - clim
    mcb_anom = mcb_ec - clim
    
  
    sigs = xr.DataArray(None, coords=dict(member=member, lat = lat, lon = lon), dims=dims, name='pval_ssp')
    sigm = xr.DataArray(None, coords=dict(member=member, lat = lat, lon = lon), dims=dims, name='pval_mcb')

    for em in range(10):
    
        index_c = bc_anom[pindices[i]].sel(member = em)
        index_e = ssp_anom[pindices[i]].sel(member = em)
        index_m = mcb_anom[pindices[i]].sel(member = em)
        foo = stats.ttest_ind(index_c, index_e).pvalue
        boo = stats.ttest_ind(index_c, index_m).pvalue
        sigs[em,:,:,] = xr.DataArray(foo)
        sigm[em,:,:,] = xr.DataArray(boo)
    
    # create a merged file of end of century ensemble mean anomalies and significances
    ssp_ec_mean = (ssp_anom.mean(dim=('year', 'member'))).to_array(name = f"SSP_{str(pindices[i])}")
    mcb_ec_mean = (mcb_anom.mean(dim=('year', 'member'))).to_array(name = f"MCB_{str(pindices[i])}")
    ssp_sig = xr.DataArray(sigs.mean(dim='member'))
    mcb_sig = xr.DataArray(sigm.mean(dim='member'))
    
    anoms_sigs = xr.merge([ssp_ec_mean, mcb_ec_mean, ssp_sig, mcb_sig])
    anoms_sigs.assign_attrs(purpose = "Ensemble mean climatology for producing figures")
    flnm = iDir + 'Anomalies_Significances_2060_' + pindices[i] + '.nc'
    anoms_sigs.to_netcdf(flnm)
    
    """
# Save the anomaly time series for future reference.
    sspnm = praw_ssp.split(pindices[i])[0] + pindices[i] + '_2050-2069_anoms.nc'
    contnm = praw_ssp.split(pindices[i])[0] + pindices[i] + '_2020-2039_anoms.nc'
    mcbnm = praw_mcb.split(pindices[i])[0] + pindices[i] + '_2050-2069_anoms.nc'
    
    ssp_anom.to_netcdf(sspnm)
    mcb_anom.to_netcdf(mcbnm)
    bc_anom.to_netcdf(contnm)
"""    

    

CDD
CWD
RX1D
N95
N99
R20mm
RX5D
PRCPTOT
SDII
R10mm


In [110]:
del(sigs)
del(sigm)

In [120]:
for i in range(len(txindices)):
    print(txindices[i])
    
    # this creates a vector, but only want first element to use as file name
    praw_ssp = glob.glob(inSSP + varnam[1] + '/*' + txindices[i] + '*.nc')[0]
    praw_mcb = glob.glob(iDir + varnam[1] + '/*' + txindices[i] + '*.nc')[0]
    
    with xr.open_dataset(praw_ssp) as ds_ssp:
        ds_ssp = ds_ssp.rename({'members':'member'})
        cont = ds_ssp.sel(year=slice('2019-01-01','2039-12-31'))
        ssp_ec = ds_ssp.sel(year=slice('2049-01-01', '2069-12-31'))
        clim = cont.mean(dim='year')
        
    with xr.open_dataset(praw_mcb) as ds_mcb:
        mcb_ec = ds_mcb.sel(year=slice('2049-01-01', '2069-12-31'))
        
    
    
    bc_anom = cont - clim
    ssp_anom = ssp_ec - clim
    mcb_anom = mcb_ec - clim
    
  
    sigs = xr.DataArray(None, coords=dict(member=member, lat = lat, lon = lon), dims=dims, name='pval_ssp')
    sigm = xr.DataArray(None, coords=dict(member=member, lat = lat, lon = lon), dims=dims, name='pval_mcb')

    for em in range(10):
    
        index_c = bc_anom[txindices[i]].sel(member = em)
        index_e = ssp_anom[txindices[i]].sel(member = em)
        index_m = mcb_anom[txindices[i]].sel(member = em)
        foo = stats.ttest_ind(index_c, index_e).pvalue
        boo = stats.ttest_ind(index_c, index_m).pvalue
        sigs[em,:,:,] = xr.DataArray(foo)
        sigm[em,:,:,] = xr.DataArray(boo)
    
    # create a merged file of end of century ensemble mean anomalies and significances
    ssp_ec_mean = (ssp_anom.mean(dim=('year', 'member'))).to_array(name = f"SSP_{str(txindices[i])}")
    mcb_ec_mean = (mcb_anom.mean(dim=('year', 'member'))).to_array(name = f"MCB_{str(txindices[i])}")
    ssp_sig = xr.DataArray(sigs.mean(dim='member'))
    mcb_sig = xr.DataArray(sigm.mean(dim='member'))
    
    anoms_sigs = xr.merge([ssp_ec_mean, mcb_ec_mean, ssp_sig, mcb_sig])
    anoms_sigs.assign_attrs(purpose = "Ensemble mean climatology for producing figures")
    flnm = iDir + 'Anomalies_Significances_2060_' + txindices[i] + '.nc'
    anoms_sigs.to_netcdf(flnm)
    


    

TXX
TXN
TX10
TX90


Daily minima Again using only the max, min and 90th/10th percentiles

In [114]:
for i in range(len(tnindices)):
    print(tnindices[i])
    
    # this creates a vector, but only want first element to use as file name
    praw_ssp = glob.glob(inSSP + varnam[2] + '/*' + tnindices[i] + '*.nc')[0]
    praw_mcb = glob.glob(iDir + varnam[2] + '/*' + tnindices[i] + '*.nc')[0]
    
    with xr.open_dataset(praw_ssp) as ds_ssp:
        ds_ssp = ds_ssp.rename({'members':'member'})
        cont = ds_ssp.sel(year=slice('2019-01-01','2039-12-31'))
        ssp_ec = ds_ssp.sel(year=slice('2049-01-01', '2069-12-31'))
        clim = cont.mean(dim='year')
        
    with xr.open_dataset(praw_mcb) as ds_mcb:
        mcb_ec = ds_mcb.sel(year=slice('2049-01-01', '2069-12-31'))
        
    
    
    bc_anom = cont - clim
    ssp_anom = ssp_ec - clim
    mcb_anom = mcb_ec - clim
    
  
    sigs = xr.DataArray(None, coords=dict(member=member, lat = lat, lon = lon), dims=dims, name='pval_ssp')
    sigm = xr.DataArray(None, coords=dict(member=member, lat = lat, lon = lon), dims=dims, name='pval_mcb')

    for em in range(10):
    
        index_c = bc_anom[tnindices[i]].sel(member = em)
        index_e = ssp_anom[tnindices[i]].sel(member = em)
        index_m = mcb_anom[tnindices[i]].sel(member = em)
        foo = stats.ttest_ind(index_c, index_e).pvalue
        boo = stats.ttest_ind(index_c, index_m).pvalue
        sigs[em,:,:,] = xr.DataArray(foo)
        sigm[em,:,:,] = xr.DataArray(boo)
    
    # create a merged file of end of century ensemble mean anomalies and significances
    ssp_ec_mean = (ssp_anom.mean(dim=('year', 'member'))).to_array(name = f"SSP_{str(txindices[i])}")
    mcb_ec_mean = (mcb_anom.mean(dim=('year', 'member'))).to_array(name = f"MCB_{str(txindices[i])}")
    ssp_sig = xr.DataArray(sigs.mean(dim='member'))
    mcb_sig = xr.DataArray(sigm.mean(dim='member'))
    
    anoms_sigs = xr.merge([ssp_ec_mean, mcb_ec_mean, ssp_sig, mcb_sig])
    anoms_sigs.assign_attrs(purpose = "Ensemble mean climatology for producing figures")
    flnm = iDir + 'Anomalies_Significances_2060_' + tnindices[i] + '.nc'
    anoms_sigs.to_netcdf(flnm)
    


    

TNX
TNN
TN10
TN90


In [115]:
anoms_sigs

<xarray.Dataset>
Dimensions:   (lat: 192, lon: 288, variable: 1, members: 10)
Coordinates:
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * variable  (variable) object 'TN90'
  * members   (members) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    SSP_TX90  (variable, lat, lon) float64 11.83 11.83 11.83 ... 5.09 5.075 5.12
    MCB_TX90  (variable, lat, lon, members) float64 1.355e+03 ... 885.0
    pval_ssp  (lat, lon) float64 0.01352 0.01352 0.01352 ... 0.1332 0.1292
    pval_mcb  (lat, lon) float64 1.409e-76 1.409e-76 ... 4.484e-74 6.86e-74